<a href="https://colab.research.google.com/github/arthur-kt/PPP-Intertemporal-Converter/blob/main/PPP_converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PPP Interporal Converter

## Parâmetros Iniciais

In [9]:
!pip install fredapi
!pip install pycountry

from fredapi import Fred
import pycountry
import pandas as pd
from datetime import datetime
import requests

# Inicialize a chave do FRED
fred = Fred(api_key='11cb1e53b57458acaddd29a16e0efe1b')

# Chave API para taxa de câmbio
exchangeratesapi = '5f749a0aa7b1501d0bec324f453a7336'

# Obtenha o CPI dos EUA para ajustar pela inflação
cpi_data = fred.get_series('CPIAUCSL')
cpi_df = cpi_data.to_frame(name='CPI')
cpi_df.index = pd.to_datetime(cpi_df.index)
cpi_df_ano = cpi_df.resample('YE').mean()  # Média anual do CPI

# CPI atual para trazer o valor a 2024
cpi_atual = cpi_df_ano.loc['2024'].values[0]

In [2]:
# Importar dados PPP
url = 'https://raw.githubusercontent.com/arthur-kt/PPP-Intertemporal-Converter/eabb3ca9caaca37dc9d9f1ae46ad03f7bc699954/ppp_att.csv'
ppp_data =  pd.read_csv(url, delimiter=',')

## Insira seus dados:

In [11]:
# Dados

## Insira em 'País_Moeda' o o país emissor da moeda utilizada (no caso de uso de moedas diferentes do país de origem), em 'País do Programa' o país de origem do Programa, em 'Ano' o ano da origem dos recursos, em 'Gasto_Local' a quantia dispendida, em 'Programa' o nome do Programa e em 'Descrição' uma breve descrição da função ou destinação dos valores.
## A coluna 'Ano2' serve para os programas que tem custos entre 2 diferentes anos, sendo necessário calcular a média do PPP e da inflação americana. Então, insira no 'Ano2' o último ano do programa, se não, preencha com 0.

dados = {
    'País_Moeda': ['US',  #1
             'US',  #2
             'US',  #3
             'US',  #4
             'US',  #5
             'KR',  #6
             'KR',  #7
             'KR',  #8
             'KR',  #9
             'KR',  #10
             'GB',  #11
             'GB',  #12
             'GB',  #13
             'GB',  #14
             'GB',  #15
             'GB',  #16
             'US',  #17
             'GB',  #18
             'GB',  #19
             'GB',  #20
             'GB',  #21
             'CN', #22
             'US', #23
             'CN', #24
             'CN', #25
             'CN', #26
             'CN', #27
             'US', #28
             'CN', #29
             'IN', #30
             'IN', #31
             'IN', #32
             'IN', #33
             'IN', #34
             'ZA', #35
             'ZA', #36
             'ZA', #37
             'MX', #38
             'MX', #39
             'MX', #40
             'MX', #41
             'MX', #42
             'MX', #43
             'MX', #44
             'MX',  #45
             'IN', #46
             'KR', #47
             'IN', #48
             'IN', #49
             'GB', #50
             'GB', #51
             ],  # Usando códigos ISO2 dos países

        'País do Programa': ['US',  #1
             'US',  #2
             'US',  #3
             'US',  #4
             'US',  #5
             'KR',  #6
             'KR',  #7
             'KR',  #8
             'KR',  #9
             'KR',  #10
             'GB',  #11
             'GB',  #12
             'GB',  #13
             'GB',  #14
             'GB',  #15
             'GB',  #16
             'GB',  #17
             'GB',  #18
             'GB',  #19
             'GB',  #20
             'GB',  #21
             'CN', #22
             'CN', #23
             'CN', #24
             'CN', #25
             'CN', #26
             'CN', #27
             'CN', #28
             'CN', #29
             'IN', #30
             'IN', #31
             'IN', #32
             'IN', #33
             'IN', #34
             'ZA', #35
             'ZA', #36
             'ZA', #37
             'MX', #38
             'MX', #39
             'MX', #40
             'MX', #41
             'MX', #42
             'MX', #43
             'MX', #44
             'MX', #45
             'IN', #46
             'KR', #47
             'IN', #48
             'IN', #49
             'GB', #50
             'GB', #51
             ],  # Usando códigos ISO2 dos países

    'Ano': [2021,  #1
            2021,  #2
            2021,  #3
            2023,  #4
            2022,  #5
            2023,  #6
            2023,  #7
            2023,  #8
            2023,  #9
            2023,  #10
            2023,  #11
            2023,  #12
            2024,  #13
            2017,  #14
            2017,  #15
            2019,  #16
            2017,  #17
            2019,  #18
            2021,  #19
            2022,  #20
            2022,  #21
            2016, #22
            2019, #23
            2009, #24
            2019, #25
            2009, #26
            2020, #27
            2008, #28
            2021, #29
            2012, #30
            2018, #31
            2023, #32
            2020, #33
            2021, #34
            2016, #35
            2012, #36
            2011, #37
            2008, #38
            2007, #39
            2006, #40
            2009, #41
            2013, #42
            2013, #43
            2021, #44
            2021, #45
            2018, #46
            2024, #47
            2021, #48
            2020, #49
            2021, #50
            2021, #51
            ],

   'Ano2': [0,  #1
            0,  #2
            0,  #3
            0,  #4
            0,  #5
            0,  #6
            0,  #7
            0,  #8
            0,  #9
            0,  #10
            0,  #11
            0,  #12
            0,  #13
            0,  #14
            0,  #15
            0,  #16
            0,  #17
            0,  #18
            0,  #19
            0,  #20
            0,  #21
            0, #22
            0, #23
            2016, #24
            0, #25
            0, #26
            0, #27
            2020, #28
            0, #29
            2017, #30
            2020, #31
            0, #32
            0, #33
            0, #34
            0, #35
            0, #36
            2015, #37
            0, #38
            0, #39
            0, #40
            0, #41
            2018, #42
            2018, #43
            0, #44
            0, #45
            2020, #46
            0, #47
            0, #48
            0, #49
            0, #50
            0, #51
            ],

    'Gasto_Local': [4460000000,  #1
                    2650000000,  #2
                    6500000000,  #3
                    2500000000,  #4
                    2105000000,  #5
                    780100000000,  #6
                    49500000000,  #7
                    1880000000000,  #8
                    60400000000,  #9
                    2100000000000,  #10
                    1100000000,  #11
                    17000000,  #12
                    7600000,  #13
                    5000000000,  #14
                    80000000000,  #15
                    4000000000,  #16
                    3000000000,  #17
                    948100000000,  #18
                    560000000,  #19
                    650000000,  #20
                    520000000,  #21
                    16000000000000, #22
                    291000000000, #23
                    5640000000000, #24
                    1861695000000, #25
                    15, #26
                    74, #27
                    3800000000, #28
                    1147000000, #29
                    1250000000, #30
                    4000000000, #31
                    3000000000, #32
                    69400000000, #33
                    5000000000, #34
                    14426000000, #35
                    1600000000, #36
                    13747152, #37
                    4500000000, #38
                    4500000000, #39
                    4000000000, #40
                    1700000000, #41
                    2138280000, #42
                    11900000, #43
                    89139000000, #44
                    44048700000, #45
                    1440000000, #46
                    55000000000, #47
                    150000000000, #48
                    34200000000, #49
                    14600000000, #50
                    800000000, #51
                    ],  # Gasto em moeda local

    'Programa': ['EOs 14017 and 14001',  #1
                 'EOs 14017 and 14001',  #2
                 'ARPA-H',  #3
                 'ARP A-H',  #4
                 'Executive Order on Advancing Biotechnology and Biomanufacturing Innovation',  #5
                 'Orçamento 2024',  #6
                 'Korean ARPA-H',  #7
                 'Korean ARPA-H',  #8
                 'Boston-Korea',  #9
                 'First National Strategic Technology Nurture Plan',  #10
                 'Medicines Manufacturing Industry Partnership (MMIP)',  #11
                 'Life Sciences Innovative Manufacturing Fund (LSIMF)',  #12
                 'Life Sciences Innovative Manufacturing Fund (LSIMF)',  #13
                 'The Life Sciences Industrial Strategy',  #14
                 'The Life Sciences Industrial Strategy',  #15
                 'The Life Sciences Industrial Strategy',  #16
                 'The Life Sciences Industrial Strategy',  #17
                 'The Life Sciences Industrial Strategy',  #18
                 'UK Innovation Strategy',  #19
                 'Life Sciences 2030 Skills Strategy',  #20
                 'Life Sciences 2030 Skills Strategy',  #21
                 'Healthy China 2030', #22
                 'Healthy China 2030', #23
                 'Healthy China 2030', #24
                 '14º Plano Quinquenal Chinês (2021-2025)', #25
                 '14º Plano Quinquenal Chinês (2021-2025)', #26
                 '14º Plano Quinquenal Chinês (2021-2025)', #27
                 '14º Plano Quinquenal para o Desenvolvimento da Bioeconomia', #28
                 'Relatório de autoavaliação sobre o desempenho geral dos fundos de subsídios regulatórios de medicamentos em 2021', #29
                 'Programa de Desenvolvimento de Clusters para o Setor Farmacêutico (CDP-PS)', #30
                 'Programa de Desenvolvimento de Clusters para o Setor Farmacêutico (CDP-PS)', #31
                 'Programa de Desenvolvimento de Clusters para o Setor Farmacêutico (CDP-PS)', #32
                 'Programas de incentivos vinculados (PLI)', #33
                 'Strengthening of Pharmaceutical Industry Scheme', #34
                 'Investimento em P&D GOV 2016', #35
                 'Industrial Policy Action Plan - Ketlaphela', #36
                 'Industrial Policy Action Plan - Ketlaphela', #37
                 'Estímulos Fiscales', #38
                 'Estímulos Fiscales', #39
                 'Estímulos Fiscales', #40
                 'Programa de Estímulos à Inovação (PEI)', #41
                 'Programa de Estímulos à Inovação (PEI', #42
                 'Programa de Estímulos à Inovação (PEI)', #43
                 'Programa Especial de Ciencia, Tecnología e Innovación 2021-2024', #44
                 'Programa Especial de Ciencia, Tecnología e Innovación 2021-2024',  #45
                 'Programa de assistência para atualização de tecnologia farmacêutica', #46
                 'Korean ARPA-H',  #47
                 'Programas de incentivos vinculados (PLI)', #48
                 'Programas de incentivos vinculados (PLI)', #49
                 'Build Back Better: Our Plan for Growth and Innovation', #50
                 'Build Back Better: Our Plan for Growth and Innovation', #51
                 ],  # Nomes dos programas

    'Descrição': ['Valores gasto',  #1
                  'Investimentos planejados',  #2
                  '2022-2024',  #3
                  '2024-2025',  #4
                  'Objective to improve and expand domestic biomanufacturing production capacity and processes',  #5
                  'Feito em 2023',  #6
                  'Alocado',  #7
                  'Entre 2024-2033',  #8
                  'Sem descrição',  #9
                  'De 2023 a 2028',  #10
                  'Follow the Green, High-Tech Road',  #11
                  'Primeira fase',  #12
                  'Segunda fase (2024)',  #13
                  '-',  #14
                  'Over the next 10 years',  #15
                  'Atingir esse nível de gasto governamental em P&D em 2027',  #16
                  'Gasto governamental em P&D em 2017',  #17
                  '-',  #18
                  '-',  #19
                  '-',  #20
                  'Entre 2025-2030',  #21
                  'De 2016 a 2030', #22
                  'Government R&D spending on biotech', #23
                  'From 2009 - 2016 to improve its healthcare system', #24
                  'Valores não relacionados ao plano: Despesas governamentais com saúde', #25
                  'Valores não relacionados ao plano: Subsídios governamentais na saúde por pessoa 2009', #26
                  'Valores não relacionados ao plano: Subsídios governamentais na saúde por pessoa 2020', #27
                  'Valores não relacionados ao plano: Investing over the period of 2008–2020 in biotechnology R&D', #28
                  'Fundos de subsídio para supervisão de alimentos e medicamentos alocados pelo governo central em 2021: valor para produtos farmacêuticos', #29
                  'Período de 2012 a 2017', #30
                  'Período de 2018 a 2020 (50% Promoção de Parques de IFAs + 50% assistência para parques de dispositivos médicos)', #31
                  'Orçamento ampliado em 2023 até o ano fiscal de 2025-2026', #32
                  'Incentivo para produção doméstica de materiais de partida, intermediários de síntese e IFAs', #33
                  'Orçamento para o período de 2021-2026', #34
                  'Gastos com P&D em 2016 (Geral, não apenas saúde)', #35
                  'Investimento total projetado em 2012', #36
                  'Financiamento de fato de 2011 a 2015', #37
                  'Programa de Estímulos Fiscales al Gasto en Investigación y Desarrollo de Tecnología de las Empresas Privadas en México', #38
                  'Programa de Estímulos Fiscales al Gasto en Investigación y Desarrollo de Tecnología de las Empresas Privadas en México', #39
                  'Programa de Estímulos Fiscales al Gasto en Investigación y Desarrollo de Tecnología de las Empresas Privadas en México', #40
                  '-', #41
                  'Destinação ao setor Farmacêutico de 2013-2018  (proxy com % de projetos do ramo farmacêutico)', #42
                  'Investigação em biotecnologia para fins não agrícolas de 2013 a 2018', #43
                  'Gasto federal en ciencia y tecnología en temas prioritarios (um deles é Saúde)', #44
                  'Gasto federal en investigación Científica y desarrollo experimental en temas prioritarios (um deles é Saúde)',  #45
                  'Período de 2018 a 2020', #46
                  'Research funding over the next 5 years', #47
                  'Programa para medicamentos', #48
                  'Programa para dispositivos médicos', #49
                  'Não específico para indústria da saúde: government investment in R&D in 2021-22', #50
                  'Não específico para indústria da saúde: funding for the Advanced Research & Invention Agency (ARIA)', #51
                  ]  # Descrições dos programas
}

#Check se todos dados estão com o mesmo número de entradas
print(len(dados['Programa']),
len(dados['Descrição']),
len(dados['País_Moeda']),
len(dados['País do Programa']),
len(dados['Ano']),
len(dados['Ano2']),
len(dados['Gasto_Local']))

df = pd.DataFrame(dados)

51 51 51 51 51 51 51


## Obter taxa de conversão do dólar 2024

In [4]:
# Função para obter o código de moeda usando o código de país ISO2
def get_currency_code(country_code):
    try:
        country = pycountry.countries.get(alpha_2=country_code)
        currency = pycountry.currencies.get(numeric=country.numeric)
        return currency.alpha_3 if currency else None
    except AttributeError:
        print(f"Não foi possível encontrar a moeda para o código de país: {country_code}")
        return None

# Função para buscar a taxa de câmbio média do ano de 2024
def get_exchange_rate_2024(currency_code):
    try:
        url = f"https://api.exchangerate-api.com/v4/latest/{currency_code}?access_key={exchangeratesapi}"

        response = requests.get(url)
        data = response.json()
        end_date = datetime.today().strftime('%Y-%m-%d')
        exchange_rate = data['rates'].get('USD')  # Taxa de câmbio para USD
        return exchange_rate
    except Exception as e:
        print(f"Erro ao obter taxa de câmbio para {currency_code}: {e}")
        return None

## Função Conversão

In [5]:
# Função para obter o PPP ajustado de um país em um ano específico a partir do CSV
def get_ppp_adjusted(country_code, year):
    ppp_row = ppp_data[(ppp_data['Country ID'] == country_code) & (ppp_data['Year'] == year)]
    return ppp_row.iloc[0]['PPP']

# Função para converter gasto local em dólar ajustado pela inflação dos EUA
def converter_para_dolar_corrente(row):
    # Para o ano 2023, usa os dados de PPP externos
    if row['Ano2'] == 0:
      if row['Ano'] == 2024:
          currency_code = get_currency_code(row['País_Moeda'])
          if currency_code:
              exchange_rate = get_exchange_rate_2024(currency_code)
              gasto_em_dolar_2024 = row['Gasto_Local'] * exchange_rate
              return gasto_em_dolar_2024
      else:
          # Para demais anos
           ppp_adjusted = get_ppp_adjusted(row['País_Moeda'], row['Ano'])
           gasto_em_dolar_ppp = row['Gasto_Local'] / ppp_adjusted

      # Ajuste pela inflação dos EUA
      cpi_ano_referencia = cpi_df_ano.loc[str(row['Ano'])].values[0]
      gasto_ajustado = gasto_em_dolar_ppp * (cpi_atual / cpi_ano_referencia)

    else:
      # Caso exista Ano2, calcula o PPP médio entre Ano1 e Ano2

      ppp_ano1 = get_ppp_adjusted(row['País_Moeda'], row['Ano'])
      ppp_ano2 = get_ppp_adjusted(row['País_Moeda'], row['Ano2'])

      # Média dos gastos enre entre Ano1 e Ano2
      gasto_em_dolar_ppp = row['Gasto_Local'] / ((ppp_ano1 + ppp_ano2) / 2)


      # Ajuste pela inflação dos EUA usando CPI de Ano1 e Ano2

      # Obter o índice CPI de ambos os anos
      cpi_ano1 = cpi_df_ano.loc[str(row['Ano'])].values[0]
      cpi_ano2 = cpi_df_ano.loc[str(row['Ano2'])].values[0]

      # Calcula a média do CPI para ajuste pela inflação
      cpi_media = (cpi_ano1 + cpi_ano2) / 2

      # Ajusta o gasto usando a média do CPI
      gasto_ajustado = gasto_em_dolar_ppp * (cpi_atual / cpi_media)

    return float(gasto_ajustado)


## Função Formatação

In [6]:
# Função para formatar valores em milhões, bilhões ou trilhões para Moeda Local
def formatar_valor_moeda_local(valor, codigo_pais):
    currency_code = get_currency_code(codigo_pais)  # Obtém o código da moeda do país
    if not currency_code:
        currency_code = "Moeda desconhecida"  # Se não houver código, exibe "Moeda desconhecida"

    if valor >= 1_000_000_000_000:  # Trilhão
        return f"{currency_code} {valor / 1_000_000_000_000:.2f} trilhões"
    elif valor >= 1_000_000_000:  # Bilhão
        return f"{currency_code} {valor / 1_000_000_000:.2f} bilhões"
    elif valor >= 1_000_000:  # Milhão
        return f"{currency_code} {valor / 1_000_000:.2f} milhões"
    else:
        return f"{currency_code} {valor:.2f}"

# Função para formatar valores em milhões, bilhões ou trilhões para Dólar Ajustado
def formatar_valor_dolar(valor):
    if valor >= 1_000_000_000_000:  # Trilhão
        return f"USD {valor / 1_000_000_000_000:.2f} trilhões"
    elif valor >= 1_000_000_000:  # Bilhão
        return f"USD {valor / 1_000_000_000:.2f} bilhões"
    elif valor >= 1_000_000:  # Milhão
        return f"USD {valor / 1_000_000:.2f} milhões"
    else:
        return f"USD {valor:.2f}"

## Conversor

In [12]:
# Aplicar a função para cada linha no DataFrame
df['Gasto_Convertido_Dolar_2024'] = df.apply(converter_para_dolar_corrente, axis=1)

# Aplicar as novas funções para as colunas 'Gasto_Local' e 'Gasto_Convertido_Dolar_2024'
df['Gasto Local'] = df.apply(lambda row: formatar_valor_moeda_local(row['Gasto_Local'], row['País_Moeda']), axis=1)
df['Gasto Convertido US$ 2024'] = df['Gasto_Convertido_Dolar_2024'].apply(formatar_valor_dolar)

# Exibir o DataFrame com as colunas formatadas
df[['País do Programa', 'Programa', 'Descrição', 'Ano', 'Ano2', 'Gasto Local', 'Gasto Convertido US$ 2024']]

<ipython-input-5-2bc86b79ebb2>:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return float(gasto_ajustado)


,País do Programa,Programa,Descrição,Ano,Ano2,Gasto Local,Gasto Convertido US$ 2024
0,US,EOs 14017 and 14001,Valores gasto,2021,0,USD 4.46 bilhões,USD 5.15 bilhões
1,US,EOs 14017 and 14001,Investimentos planejados,2021,0,USD 2.65 bilhões,USD 3.06 bilhões
2,US,ARPA-H,2022-2024,2021,0,USD 6.50 bilhões,USD 7.50 bilhões
3,US,ARP A-H,2024-2025,2023,0,USD 2.50 bilhões,USD 2.57 bilhões
4,US,Executive Order on Advancing Biotechnology and...,Objective to improve and expand domestic bioma...,2022,0,USD 2.10 bilhões,USD 2.25 bilhões
5,KR,Orçamento 2024,Feito em 2023,2023,0,KRW 780.10 bilhões,USD 1.00 bilhões
6,KR,Korean ARPA-H,Alocado,2023,0,KRW 49.50 bilhões,USD 63.48 milhões
7,KR,Korean ARPA-H,Entre 2024-2033,2023,0,KRW 1.88 trilhões,USD 2.41 bilhões
8,KR,Boston-Korea,Sem descrição,2023,0,KRW 60.40 bilhões,USD 77.46 milhões
9,KR,First National Strategic Technology Nurture Plan,De 2023 a 2028,2023,0,KRW 2.10 trilhões,USD 2.69 bilhões
